In [ ]:
#Importing all the necessary libraries for our model
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
data=pd.read_csv("/content/traffic.csv", skiprows=range(1, 25340))
data

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,Unnamed: 15,hour,month_day,year
0,7/29/2016 19:00,0,289,77,6,250,8,8,297.31,0.0,0,20,Clouds,few clouds,3526,4239,19,29,2016
1,7/29/2016 20:00,0,287,77,8,260,8,8,295.43,0.0,0,40,Clouds,scattered clouds,2658,3526,20,29,2016
2,7/29/2016 21:00,0,180,76,7,270,6,6,292.85,0.0,0,40,Clouds,scattered clouds,2794,2658,21,29,2016
3,7/29/2016 22:00,0,188,72,9,280,3,3,290.84,0.0,0,40,Clouds,scattered clouds,2651,2794,22,29,2016
4,7/29/2016 23:00,0,186,63,7,270,4,4,290.45,0.0,0,75,Clouds,broken clouds,3206,2651,23,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,5/17/2017 21:00,0,74,85,1,328,7,7,287.88,0.0,0,90,Mist,mist,2348,2348,21,17,2017
8401,5/17/2017 22:00,0,81,70,1,24,2,2,286.95,0.0,0,90,Rain,heavy intensity rain,2194,2348,22,17,2017
8402,5/17/2017 22:00,0,138,70,1,24,1,1,286.95,0.0,0,90,Mist,mist,2194,2194,22,17,2017
8403,5/17/2017 23:00,0,229,66,1,42,6,6,285.75,0.0,0,90,Rain,heavy intensity rain,1328,2194,23,17,2017


In [ ]:
data.info()
type(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            8405 non-null   object 
 1   is_holiday           8405 non-null   int64  
 2   air_pollution_index  8405 non-null   int64  
 3   humidity             8405 non-null   int64  
 4   wind_speed           8405 non-null   int64  
 5   wind_direction       8405 non-null   int64  
 6   visibility_in_miles  8405 non-null   int64  
 7   dew_point            8405 non-null   int64  
 8   temperature          8405 non-null   float64
 9   rain_p_h             8405 non-null   float64
 10  snow_p_h             8405 non-null   int64  
 11  clouds_all           8405 non-null   int64  
 12  weather_type         8405 non-null   object 
 13  weather_description  8405 non-null   object 
 14  traffic_volume       8405 non-null   int64  
 15  Unnamed: 15          8405 non-null   i

pandas.core.frame.DataFrame

In [ ]:

null_values = data.isnull().sum()
null_values

date_time              0
is_holiday             0
air_pollution_index    0
humidity               0
wind_speed             0
wind_direction         0
visibility_in_miles    0
dew_point              0
temperature            0
rain_p_h               0
snow_p_h               0
clouds_all             0
weather_type           0
weather_description    0
traffic_volume         0
Unnamed: 15            0
hour                   0
month_day              0
year                   0
dtype: int64

In [ ]:
#Removing unnecessary features that are not relevant for traffic prediction
features_to_drop = ['wind_speed', 'wind_direction','rain_p_h','snow_p_h','weather_type','date_time']
data.drop(columns=features_to_drop, inplace=True)
data

,is_holiday,air_pollution_index,humidity,visibility_in_miles,dew_point,temperature,clouds_all,weather_description,traffic_volume,Unnamed: 15,hour,month_day,year
0,0,289,77,8,8,297.31,20,few clouds,3526,4239,19,29,2016
1,0,287,77,8,8,295.43,40,scattered clouds,2658,3526,20,29,2016
2,0,180,76,6,6,292.85,40,scattered clouds,2794,2658,21,29,2016
3,0,188,72,3,3,290.84,40,scattered clouds,2651,2794,22,29,2016
4,0,186,63,4,4,290.45,75,broken clouds,3206,2651,23,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,0,74,85,7,7,287.88,90,mist,2348,2348,21,17,2017
8401,0,81,70,2,2,286.95,90,heavy intensity rain,2194,2348,22,17,2017
8402,0,138,70,1,1,286.95,90,mist,2194,2194,22,17,2017
8403,0,229,66,6,6,285.75,90,heavy intensity rain,1328,2194,23,17,2017


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
data['weather_description_encoded'] = ordinal_encoder.fit_transform(data[['weather_description']])


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# data['weather_description_encoded'] = label_encoder.fit_transform(data['weather_description'])

In [ ]:
data

,is_holiday,air_pollution_index,humidity,visibility_in_miles,dew_point,temperature,clouds_all,weather_description,traffic_volume,Unnamed: 15,hour,month_day,year,weather_description_encoded
0,0,289,77,8,8,297.31,20,few clouds,3526,4239,19,29,2016,3.0
1,0,287,77,8,8,295.43,40,scattered clouds,2658,3526,20,29,2016,23.0
2,0,180,76,6,6,292.85,40,scattered clouds,2794,2658,21,29,2016,23.0
3,0,188,72,3,3,290.84,40,scattered clouds,2651,2794,22,29,2016,23.0
4,0,186,63,4,4,290.45,75,broken clouds,3206,2651,23,29,2016,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,0,74,85,7,7,287.88,90,mist,2348,2348,21,17,2017,16.0
8401,0,81,70,2,2,286.95,90,heavy intensity rain,2194,2348,22,17,2017,8.0
8402,0,138,70,1,1,286.95,90,mist,2194,2194,22,17,2017,16.0
8403,0,229,66,6,6,285.75,90,heavy intensity rain,1328,2194,23,17,2017,8.0


In [ ]:
# Defining the features (i.e,input variables) and the sole target (i.e,traffic volume)
features = ['is_holiday','air_pollution_index','humidity','temperature','visibility_in_miles' ,'dew_point','clouds_all','weather_description_encoded','hour','month_day','month_day','Unnamed: 15']
target = ['traffic_volume']
# Extract features (X) and target (y) from the data
X = data[features]
y = data[target]

In [ ]:
#Scale the features and target using MinMaxScaler to normalize the data
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y).flatten()
X
y


array([0.48426982, 0.36502267, 0.38370655, ..., 0.30127765, 0.18230526,
       0.18230526])

In [ ]:
# Here we are simply Spliting the data into training and testing sets for model evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Training a Multi-Layer Perceptron (MLP) Regressor model
# mlp_regressor = MLPRegressor(random_state=1, max_iter=500)
mlp_regressor = MLPRegressor(random_state=1, max_iter=500)
mlp_regressor.fit(X_train, y_train)
num_hidden_layers = mlp_regressor.n_layers_ - 2
print("Number of hidden layers:", num_hidden_layers)

y_pred = mlp_regressor.predict(X_test)

Number of hidden layers: 1


In [ ]:
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, y_pred)
print("R-squared (R2):", r_squared)



Mean Absolute Error: 0.05743752271316719
R-squared (R2): 0.9086682416711769


In [ ]:
print(type(y_train))

<class 'numpy.ndarray'>


In [ ]:
y_train = np.array(y_train)

In [ ]:
gbm_regr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gbm_regr.fit(X_train, y_train.ravel())
gbm_predictions = gbm_regr.predict(X_test)
gbm_mae = mean_absolute_error(y_test, gbm_predictions)
print("GBM Mean Absolute Error:", gbm_mae)
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, gbm_predictions)
print("R-squared (R2):", r_squared)


GBM Mean Absolute Error: 0.03554588735230978
R-squared (R2): 0.9670050361106475


In [ ]:
gbm_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4)
mlp_model = MLPRegressor(hidden_layer_sizes=(100,))

# Initialize stacking ensemble
estimators = [('gbm', gbm_model), ('mlp', mlp_model)]
stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

# Train stacking ensemble
stacking_regressor.fit(X_train, y_train)

# Evaluation of final performance
stacking_predictions = stacking_regressor.predict(X_test)
stacking_mae = mean_absolute_error(y_test, stacking_predictions)
print("Stacking Ensemble Mean Absolute Error:", stacking_mae)
from sklearn.metrics import r2_score
r_squared = r2_score(y_test, stacking_predictions)
print("R-squared (R2):", r_squared)

Stacking Ensemble Mean Absolute Error: 0.02978521382412214
R-squared (R2): 0.9754767142671463


**Here we are taking input values from the user and using it to calculate the Traffic volume**

In [ ]:

# Taking inputs from the user
print("\nEnter values for the following features:")
is_holiday = float(input("Is holiday (0 for No, 1 for Yes): "))
air_pollution_index = float(input("Air pollution index: "))
humidity = float(input("Humidity(in %): "))
temperature= float(input("Temperature: "))
visibility_in_miles=float(input("visibility_in_miles:"))
dew_point= float(input("dew_point:"))
clouds_all=float(input("clouds_all(in %):"))
hour=float(input("hour:"))
month_day=float(input("month_day:"))

# Create a DataFrame with all features
input_data_all_features = pd.DataFrame({
    'is_holiday': [is_holiday],
    'air_pollution_index': [air_pollution_index],
    'humidity': [humidity],
    'temperature': [temperature],
    'visibility_in_miles': [visibility_in_miles],
    'dew_point': [dew_point],
    'clouds_all': [clouds_all],
    'weather_description_encoded': [3.0],
    'hour': [hour],
    'month_day': [month_day],
    'Unnamed: 15':[5181]
}, columns=features)
# Scale input features using the same scaler trained on X_train
input_data_scaled = x_scaler.transform(input_data_all_features)

# # Predicted the value of traffic volume using MLP model
predicted_traffic_mlp = mlp_regressor.predict(input_data_scaled)
predicted_traffic_mlp = y_scaler.inverse_transform(predicted_traffic_mlp.reshape(-1, 1))[0][0]
print("Predicted 2traffic volume (MLP):", predicted_traffic_mlp)

# # Predicted the value of  traffic volume using GBM model

predicted_traffic_gbm = gbm_regr.predict(input_data_scaled)
predicted_traffic_gbm = y_scaler.inverse_transform(predicted_traffic_gbm.reshape(-1, 1))[0][0]
print("Predicted traffic volume (GBM):", predicted_traffic_gbm)

# Predicted the value of traffic volume using stacking ensemble
predicted_traffic_stacking = stacking_regressor.predict(input_data_scaled)
predicted_traffic_stacking = y_scaler.inverse_transform(predicted_traffic_stacking.reshape(-1, 1))[0][0]
print("Predicted traffic volume (Stacking Ensemble):", predicted_traffic_stacking)



Enter values for the following features:
Is holiday (0 for No, 1 for Yes): 1
Air pollution index: 190
Humidity(in %): 56
Temperature: 180
visibility_in_miles:2
dew_point:20
clouds_all(in %):45
hour:3
month_day:4
Predicted 2traffic volume (MLP): 2718.2998099703714
Predicted traffic volume (GBM): 2976.7882945385827
Predicted traffic volume (Stacking Ensemble): 2845.7767919790563


**Visualizing the data we are working on**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Displaying Histograms
data.hist(figsize=(12, 8))
plt.suptitle('Histograms of Numeric Features')
plt.tight_layout()
plt.show()


# Line Plot (hour vs traffic volume)
plt.figure(figsize=(10, 6))
sns.lineplot(x='hour', y='traffic_volume', data=data)
plt.title('Traffic Volume Trend by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Traffic Volume')
plt.show()



# Bar Chart (holiday status vs traffic volume)
plt.figure(figsize=(8, 6))
sns.barplot(x='is_holiday', y='traffic_volume', data=data)
plt.title('Average Traffic Volume on Holidays vs Non-Holidays')
plt.xlabel('Is Holiday')
plt.ylabel('Average Traffic Volume')
plt.xticks([0, 1], ['No', 'Yes'])
plt.show()

# Box Plot (hour vs traffic volume)

plt.figure(figsize=(10, 6))
sns.boxplot(x='hour', y='traffic_volume', data=data)
plt.title('Traffic Volume Distribution by Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Traffic Volume')
plt.show()


**Comparing the models to find the best fit**

In [ ]:
#Here we are determining the most accurate model and also printing the Mean Absolute Error of the same.
mlp_regressor = MLPRegressor(random_state=1, max_iter=500)
mlp_regressor.fit(X_train, y_train)
y_pred_mlp = mlp_regressor.predict(X_test)


gbm_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gbm_regressor.fit(X_train, y_train)
gbm_predictions = gbm_regressor.predict(X_test)


stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stacking_regressor.fit(X_train, y_train)
stacking_predictions = stacking_regressor.predict(X_test)

# Calculating MAE values
mae_mlp = mean_absolute_error(y_test, y_pred_mlp)
mae_gbm = mean_absolute_error(y_test, gbm_predictions)
mae_stacking = mean_absolute_error(y_test, stacking_predictions)

# Storing the MAE values in a dictionary
mae_values = {
    'MLP': mae_mlp,
    'GBM': mae_gbm,
    'Stacking Ensemble': mae_stacking
}

# Finding the model with the lowest MAE (most accurate)
best_model = min(mae_values, key=mae_values.get)

# Printing the most accurate model
print("The most accurate model for traffic volume prediction is:", best_model)
print("Mean Absolute Error for the most accurate model:", mae_values[best_model])




**DISPLAYING TRAFFIC LEVELS AS HIGH, MODERATE AND LOW:**


In [ ]:
# Define the ranges for traffic levels
low_traffic_range = (0, 3000)
moderate_traffic_range = (3000, 5500)
high_traffic_range = (5500, float('inf'))  # Use float('inf') for infinity

# Assuming you have calculated the predicted traffic volume
predicted_traffic = 4500  # Example predicted traffic volume

# Check which range the predicted traffic volume falls into
if predicted_traffic >= low_traffic_range[0] and predicted_traffic < low_traffic_range[1]:
    traffic_level = "Low Traffic"
elif predicted_traffic >= moderate_traffic_range[0] and predicted_traffic < moderate_traffic_range[1]:
    traffic_level = "Moderate Traffic"
else:
    traffic_level = "High Traffic"

# Print the predicted traffic level
print("Predicted Traffic Level:", traffic_level)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns



# Define the models and their respective MAE values
models = ['MLP', 'GBM', 'Stacking Ensemble']
mae_values = [mae_mlp, mae_gbm, mae_stacking]

# Create a DataFrame for plotting
df = pd.DataFrame({'Models': models, 'MAE': mae_values})

# Create a bar plot for model comparison
plt.figure(figsize=(8, 6))
sns.barplot(data=df, x='Models', y='MAE', palette='viridis')
plt.xlabel('Models')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Comparison of Models for Traffic Volume Prediction')
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()
